In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Create a multi-layer perceptron neural network model to predict on a labeled dataset of your choosing. Compare this model to either a boosted tree or a random forest model and describe the relative tradeoffs between complexity and accuracy. Be sure to vary the hyperparameters of your MLP!

In [2]:

#import data of students in a math course
df1=pd.read_csv("C:\\Users\\david\\Downloads\\student-mat.csv")

#import data of students in a Portugese language course
df2=pd.read_csv("C:\\Users\\david\\Downloads\\student-por.csv")

In [3]:
#Create a combined dataframe from the math and language course

combined = [df1, df2]

combined=pd.concat(combined)

## Cleaning the Data

 Some students appear in both courses so we must delete any duplicate student

In [4]:
combined=combined.drop_duplicates(["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"])

In [5]:
combined.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [6]:
#Keeping only the continuous data
continuous_data = combined.loc[:, ['age', 'famrel', 'freetime', 'goout', 'Dalc'
                                   , 'Walc', 'health', 'G1', 'G2', 'G3',
                                  'Medu','Fedu','traveltime','studytime']]

In [7]:
continuous_data

,age,famrel,freetime,goout,Dalc,Walc,health,G1,G2,G3,Medu,Fedu,traveltime,studytime
0,18,4,3,4,1,1,3,5,6,6,4,4,2,2
1,17,5,3,3,1,1,3,5,5,6,1,1,1,2
2,15,4,3,2,2,3,3,7,8,10,1,1,1,2
3,15,3,2,2,1,1,5,15,14,15,4,2,1,3
4,16,4,3,2,1,2,5,6,10,10,3,3,1,2
5,16,5,4,2,1,2,5,15,15,15,4,3,1,2
6,16,4,4,4,1,1,3,12,12,11,2,2,1,2
7,17,4,1,4,1,1,1,6,5,6,4,4,2,2
8,15,4,2,2,1,1,1,16,18,19,3,2,1,2
9,15,5,5,1,1,1,5,14,15,15,3,4,1,2


In [8]:
exclude = ['Walc']

y = continuous_data.Walc
X = continuous_data.drop(['Walc'])

## Perform some PCA

In [9]:

# Your code here.# Your c 
# lets attempt a PCA on (hopefully) a smaller number of features

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

# scale the input into mean 0 and standardized variance
scaled_X = StandardScaler().fit_transform(X)

pca = PCA(n_components=10)
newX = pca.fit_transform(scaled_X)

print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.21996725 0.14075273 0.11274899 0.08894577 0.07508076 0.07120231
 0.06283795 0.06097226 0.05382615 0.04380175]
[45.15148407 36.11781125 32.32580405 28.71148559 26.37894007 25.68857766
 24.13259425 23.77164043 22.33519016 20.14831606]


## Random Forest

In [10]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestRegressor()


rfc.fit(newX, y)
y_1_predictions = rfc.predict(newX)


print("\nAccuracy of training set: {:.3f}".format(rfc.score(newX, y)))
print("\nAccuracy of test set: {:.3f}".format(rfc.score(newX, y)))



Accuracy of training set: 0.958

Accuracy of test set: 0.958


In [11]:
import time
start_time = time.time()
rfc = ensemble.RandomForestRegressor()


cv_score = cross_val_score(rfc, newX, y, cv=5)
print(cv_score)
print("Random Forest takes %.3f seconds" % (time.time() - start_time))

[0.7061696  0.80347718 0.7030528  0.74307591 0.72713564]
Random Forest takes 0.280 seconds


## MLP Regressor

In [12]:

# Import the model.# Import 
from sklearn.neural_network import MLPRegressor

# Establish and fit the model, with a single, 100 perceptron layer.
mlp = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000)
mlp.fit(newX, y)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [13]:
mlp.score(newX, y)

0.9473605066409946

In [14]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, newX, y, cv=5)

array([0.88923269, 0.8995617 , 0.8612193 , 0.81911754, 0.82272872])

## Conclusions
The MLP performed slightly lower than the RFC in terms of accuracy. 